In [2]:
import pandas as pd
import numpy as np
import unicodedata
from pathlib import Path
import cvxpy as cp

# --------------------------------------------------------
# 1. Parameters — tweak these for scenario testing
# --------------------------------------------------------

DATA_DIR = Path("/Users/jjsmith1701/Documents/Angels Offseason Datasets") 
PAYROLL_CEILING = 210_000_000  # realistic Angels 2026 CBT budget (tweak as you like)
MAX_TRADES = 3                 # how many trade acquisitions you allow
MIN_PITCHERS = 12              # min pitchers on 26-man
MAX_PITCHERS = 14              # max pitchers on 26-man
MIN_SP = 5                     # min starting pitchers
MIN_CATCHERS = 2               # min catchers
MIN_OUTFIELDERS = 4            # min OF
MIN_MIF = 2                    # min middle infielders (2B/SS)

# --------------------------------------------------------
# 2. Helper: normalize player names for merging with FG
# --------------------------------------------------------

def normalize_name(s: str) -> str | None:
    if not isinstance(s, str):
        return None
    s = s.strip()
    # strip basic suffixes
    for suf in [", Jr.", " Jr.", " Jr", " II", " III", " IV"]:
        if s.endswith(suf):
            s = s[: -len(suf)]
    # strip accents
    s = "".join(
        c for c in unicodedata.normalize("NFKD", s)
        if not unicodedata.combining(c)
    )
    s = s.replace(".", "")
    s = " ".join(s.split())
    return s.lower()


def attach_fg_war(df: pd.DataFrame,
                  name_col: str,
                  fg_hit: pd.DataFrame,
                  fg_pit: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    out["name_key"] = out[name_col].apply(normalize_name)

    fg_hit_ = fg_hit[["name_key", "WAR"]].rename(columns={"WAR": "WAR_hit_fg"})
    fg_pit_ = fg_pit[["name_key", "WAR"]].rename(columns={"WAR": "WAR_pit_fg"})

    out = out.merge(fg_hit_, on="name_key", how="left")
    out = out.merge(fg_pit_, on="name_key", how="left")

    return out


def compute_fg_total_war(df: pd.DataFrame,
                         pos_col: str,
                         group_col: str | None = None) -> pd.DataFrame:
    out = df.copy()

    # classify pitchers by group if available, else by position string
    if group_col is not None and group_col in out.columns:
        out["is_pitcher"] = out[group_col].isin(["SP", "RP"])
        out["is_sp"] = out[group_col].eq("SP")
        out["is_rp"] = out[group_col].eq("RP")
    else:
        pos_str = out[pos_col].astype(str)
        out["is_sp"] = pos_str.str.contains("SP")
        out["is_rp"] = pos_str.str.contains("RP")
        out["is_pitcher"] = out["is_sp"] | out["is_rp"]

    # catchers / infielders / outfielders
    pos_str = out[pos_col].astype(str)
    out["is_c"] = pos_str.str.contains(r"\bC\b")
    out["is_1b"] = pos_str.str.contains("1B")
    out["is_2b"] = pos_str.str.contains("2B")
    out["is_3b"] = pos_str.str.contains("3B")
    out["is_ss"] = pos_str.str.contains("SS")
    out["is_lf"] = pos_str.str.contains("LF")
    out["is_cf"] = pos_str.str.contains("CF")
    out["is_rf"] = pos_str.str.contains("RF")

    out["is_inf"] = out[["is_1b", "is_2b", "is_3b", "is_ss"]].any(axis=1)
    out["is_of"] = out[["is_lf", "is_cf", "is_rf"]].any(axis=1)

    # choose appropriate FG WAR
    # pitchers: use pitching WAR; hitters: use hitting WAR
    out["fg_war"] = np.where(
        out["is_pitcher"],
        out["WAR_pit_fg"],
        out["WAR_hit_fg"],
    )

    # robust fallback
    out["fg_war"] = (
        out["fg_war"]
        .fillna(out["WAR_hit_fg"])
        .fillna(out["WAR_pit_fg"])
        .fillna(0.0)
    )

    return out


# --------------------------------------------------------
# 3. Load data
# --------------------------------------------------------

angels = pd.read_csv(DATA_DIR / "Angels_2026_Master_clean.csv")
fa = pd.read_csv(DATA_DIR / "Free_Agents_2026_clean.csv")
trades = pd.read_csv(DATA_DIR / "Trade_Candidates_2026_clean.csv")
fg_hit = pd.read_csv(DATA_DIR / "fangraphs-MLBhitter_2026projections.csv")
fg_pit = pd.read_csv(DATA_DIR / "fangraphs-MLBpitcher_2026projections.csv")

# prepare Fangraphs name keys
fg_hit["name_key"] = fg_hit["NameASCII"].apply(normalize_name)
fg_pit["name_key"] = fg_pit["NameASCII"].apply(normalize_name)

# --------------------------------------------------------
# 4. Overlay Fangraphs WAR onto each pool
# --------------------------------------------------------

angels_fg = attach_fg_war(angels, "name", fg_hit, fg_pit)
fa_fg = attach_fg_war(fa, "Player", fg_hit, fg_pit)
trades_fg = attach_fg_war(trades, "player", fg_hit, fg_pit)

angels_fg = compute_fg_total_war(angels_fg, pos_col="pos_spotrac", group_col="group")
fa_fg = compute_fg_total_war(fa_fg, pos_col="Position", group_col=None)
trades_fg = compute_fg_total_war(trades_fg, pos_col="Pos", group_col=None)

# --------------------------------------------------------
# 5. Clean up salaries & filter candidate pools
# --------------------------------------------------------

# Angels salaries: use luxury_tax_salary
angels_fg["salary"] = angels_fg["luxury_tax_salary"].fillna(0.0)

# FAs: use AAV_cost, enforce a minimum (avoid 0-cost high-WAR unicorns)
MIN_FA_SAL = 1_000_000.0
fa_fg["salary"] = fa_fg["AAV_cost"].fillna(0.0)
fa_fg.loc[fa_fg["salary"] < MIN_FA_SAL, "salary"] = MIN_FA_SAL
# FA qualifying offer tag
fa_fg["is_QO"] = fa_fg["Player"].str.contains("QO", case=False, na=False).astype(float)

# Trades: use TAX_SALARY, drop guys with missing salary (or set min)
trades_fg["salary"] = trades_fg["TAX_SALARY"].fillna(0.0)
trades_fg = trades_fg[trades_fg["salary"] > 0.0].copy()


# -----------------------------
# Trade tiers — realism control
# -----------------------------

tier1 = [
    "Freddy Peralta", "Steven Kwan", "Joe Ryan", "MacKenzie Gore", "Byron Buxton",
    "Oneil Cruz", "Luis Severino", "Luis Castillo", "Brady Singer", "Jazz Chisholm",
    "Yandy Diaz", "Edward Cabrera", "Ozzie Albies"
]

tier2 = [
    "Brendan Donovan", "Paul Blackburn", "Ryan Jeffers", "Sonny Gray", "Drew Rasmussen",
    "Nolan Arenado", "Mitch Keller", "Luis Robert Jr.", "Pablo Lopez", "Randy Arozarena",
    "Bryan Reynolds", "Kyle Freeland", "Adolis Garcia", "Jeff McNeil", "Brandon Lowe",
    "Sandy Alcantara", "Bailey Ober", "Alec Bohm", "Jeffrey Springs", "Ryan Mountcastle",
    "Lars Nootbaar", "Anthony Volpe", "J.J. Bleday", "Jasson Dominguez", "Wilyer Abreu"
]

tier3 = [
    "Christian Walker", "Nick Castellanos", "Mark Vientos", "Luisangel Acuna",
    "Alek Thomas", "Jake Burger", "JoJo Romero", "Carlos Estevez",
    "Jonah Heim", "Nolan Gorman", "Josh Jung"
]

def clean_name_for_match(s: str) -> str:
    if not isinstance(s, str):
        return ""
    s = s.strip()
    s = "".join(
        c for c in unicodedata.normalize("NFKD", s)
        if not unicodedata.combining(c)
    )
    s = s.replace(".", "")
    s = " ".join(s.split())
    return s.lower()

trades_fg["player_clean"] = trades_fg["player"].apply(clean_name_for_match)

tier1_clean = [clean_name_for_match(n) for n in tier1]
tier2_clean = [clean_name_for_match(n) for n in tier2]
tier3_clean = [clean_name_for_match(n) for n in tier3]

trades_fg["trade_cost"] = 0.0
trades_fg.loc[trades_fg["player_clean"].isin(tier1_clean), "trade_cost"] = 3.0
trades_fg.loc[trades_fg["player_clean"].isin(tier2_clean), "trade_cost"] = 2.0
trades_fg.loc[trades_fg["player_clean"].isin(tier3_clean), "trade_cost"] = 1.0

# --------------------------------------------------------
# 6. Build unified player pool
# --------------------------------------------------------

# subset columns for each group
angels_pool = pd.DataFrame({
    "name": angels_fg["name"],
    "source": "ANGELS",
    "pos": angels_fg["pos_spotrac"],
    "salary": angels_fg["salary"],
    "fg_war": angels_fg["fg_war"],
    "is_pitcher": angels_fg["is_pitcher"],
    "is_sp": angels_fg["is_sp"],
    "is_rp": angels_fg["is_rp"],
    "is_c": angels_fg["is_c"],
    "is_inf": angels_fg["is_inf"],
    "is_of": angels_fg["is_of"],
    "is_2b": angels_fg["is_2b"],
    "is_ss": angels_fg["is_ss"],
    "is_3b": angels_fg["is_3b"],
    "is_cf": angels_fg["is_cf"],

})

fa_pool = pd.DataFrame({
    "name": fa_fg["Player"],
    "source": "FA",
    "pos": fa_fg["Position"],
    "salary": fa_fg["salary"],
    "fg_war": fa_fg["fg_war"],
    "is_pitcher": fa_fg["is_pitcher"],
    "is_sp": fa_fg["is_sp"],
    "is_rp": fa_fg["is_rp"],
    "is_c": fa_fg["is_c"],
    "is_inf": fa_fg["is_inf"],
    "is_of": fa_fg["is_of"],
    "is_2b": fa_fg["is_2b"],
    "is_ss": fa_fg["is_ss"],
    "is_3b": fa_fg["is_3b"],
    "is_cf": fa_fg["is_cf"],
    "is_QO": fa_fg["is_QO"],
})

trade_pool = pd.DataFrame({
    "name": trades_fg["player"],
    "source": "TRADE",
    "pos": trades_fg["Pos"],
    "salary": trades_fg["salary"],
    "fg_war": trades_fg["fg_war"],
    "is_pitcher": trades_fg["is_pitcher"],
    "is_sp": trades_fg["is_sp"],
    "is_rp": trades_fg["is_rp"],
    "is_c": trades_fg["is_c"],
    "is_inf": trades_fg["is_inf"],
    "is_of": trades_fg["is_of"],
    "is_2b": trades_fg["is_2b"],
    "is_ss": trades_fg["is_ss"],
    "is_3b": trades_fg["is_3b"],
    "is_cf": trades_fg["is_cf"],
    "trade_cost": trades_fg["trade_cost"],

})

angels_pool["is_QO"] = 0.0
trade_pool["is_QO"] = 0.0

angels_pool["trade_cost"] = 0.0
fa_pool["trade_cost"] = 0.0


# --------------------------------------------------------
# 6b. Build unified player pool (and drop self-duplicates)
# --------------------------------------------------------

# Remove FA / TRADE entries for players already on the Angels
angels_names = set(angels_pool["name"])

fa_pool = fa_pool[~fa_pool["name"].isin(angels_names)].copy()
trade_pool = trade_pool[~trade_pool["name"].isin(angels_names)].copy()

players = pd.concat([angels_pool, fa_pool, trade_pool], ignore_index=True)
players["player_id"] = np.arange(len(players))


# identify indices by group
is_angels = players["source"].eq("ANGELS").values
is_fa = players["source"].eq("FA").values
is_trade = players["source"].eq("TRADE").values

# ------------------------------------
# Block external DH-only acquisitions
# ------------------------------------
pos_str = players["pos"].astype(str)

# Any player listed with DH in their position, who is NOT already on the Angels
external_dh_mask = pos_str.str.contains("DH") & ~is_angels

idx_pitchers = np.where(players["is_pitcher"].values)[0]
idx_sp = np.where(players["is_sp"].values)[0]
idx_rp = np.where(players["is_rp"].values)[0]
idx_c = np.where(players["is_c"].values)[0]
idx_of = np.where(players["is_of"].values)[0]
idx_mif = np.where(players[["is_2b", "is_ss"]].any(axis=1).values)[0]
idx_ss = np.where(players["is_ss"].values)[0]
idx_external_cf = np.where(players["is_cf"].values & ~is_angels)[0]
idx_external_3b = np.where(players["is_3b"].values & ~is_angels)[0]
idx_external_dh = np.where(external_dh_mask.values)[0]
idx_trades = np.where(is_trade)[0]

salary = players["salary"].values
fg_war = players["fg_war"].values

# Angels base CBT payroll (already committed)
base_payroll = angels_pool["salary"].sum()
print(f"Base Angels CBT payroll from roster file: ${base_payroll:,.0f}")

# Extra payroll should only count NEW acquisitions (FA + TRADE), not Angels again.
salary_extra = salary.copy()
salary_extra[is_angels] = 0.0  # zero out Angels salaries for the extra-payroll calc

is_qo = players["is_QO"].values

trade_cost = players["trade_cost"].values
is_trade = players["source"].eq("TRADE").values


# --------------------------------------------------------
# 7. Build and solve the optimization model (CVXPY)
# --------------------------------------------------------

N = len(players)

# Decision vars
signed = cp.Variable(N, boolean=True)  # whether we control/pay the player
roster = cp.Variable(N, boolean=True)  # whether on 26-man roster

constraints = []

# Angels are already under team control; we can't "unsigned" them
constraints.append(signed[is_angels] == 1)

# You can only roster players you signed / control
constraints.append(roster <= signed)

# 26-man roster
constraints.append(cp.sum(roster) == 26)

# Core Angels
core_names = ["Mike Trout", "Zach Neto", "Reid Detmers", "Jose Soriano", "Yusei Kikuchi", "Oswald Peraza", "Jo Adell"]  # edit as you want
core_mask = players["name"].isin(core_names).values
core_catchers = ["Logan O'Hoppe", "Travis d'Arnaud"]

core_c_mask = players["name"].isin(core_catchers).values
other_c_mask = players["is_c"].values & ~core_c_mask

# Force core catchers to be signed and on the 26-man
if core_c_mask.any():
    constraints.append(signed[core_c_mask] == 1)
    constraints.append(roster[core_c_mask] == 1)

# No other catchers on the 26-man roster
if other_c_mask.any():
    constraints.append(roster[other_c_mask] == 0)

# This automatically makes 26-man have exactly 2 catchers, as long as both exist in the pool


# force them signed + on roster
constraints.append(roster[core_mask] == 1)
constraints.append(signed[core_mask] == 1)


# Pitching staff size
constraints.append(cp.sum(roster[idx_pitchers]) >= MIN_PITCHERS)
constraints.append(cp.sum(roster[idx_pitchers]) <= MAX_PITCHERS)
MAX_SP = 7      # at most 7 starting pitchers on the 26-man
MIN_RP = 5      # at least 5 relievers
external_sp_mask = players["is_sp"].values & ~is_angels
idx_external_sp = np.where(external_sp_mask)[0]

if len(idx_external_sp) > 0:
    constraints.append(cp.sum(roster[idx_external_sp]) >= 1)



# Minimum SPs, catchers, OF, MI
if len(idx_sp) > 0:
    constraints.append(cp.sum(roster[idx_sp]) >= MIN_SP)
    # Limit number of starters; require some relievers
if len(idx_sp) > 0:
    constraints.append(cp.sum(roster[idx_sp]) <= MAX_SP)

if len(idx_rp) > 0:
    constraints.append(cp.sum(roster[idx_rp]) >= MIN_RP)


if len(idx_c) > 0:
    constraints.append(cp.sum(roster[idx_c]) >= MIN_CATCHERS)

if len(idx_of) > 0:
    constraints.append(cp.sum(roster[idx_of]) >= MIN_OUTFIELDERS)

if len(idx_mif) > 0:
    constraints.append(cp.sum(roster[idx_mif]) >= MIN_MIF)

if len(idx_external_cf) > 0:
    constraints.append(cp.sum(roster[idx_external_cf]) == 1)

if len(idx_external_3b) > 0:
    constraints.append(cp.sum(roster[idx_external_3b]) == 1)

# --- Max 2 shortstops on the 26-man roster ---
if len(idx_ss) > 0:
    constraints.append(cp.sum(roster[idx_ss]) <= 2)

if len(idx_external_dh) > 0:
    # Never acquire/sign DH-only bats from outside
    constraints.append(signed[idx_external_dh] == 0)


# Payroll constraint: base Angels CBT + new acquisitions must stay under budget
extra_payroll = salary_extra @ signed  # elementwise dot product
total_payroll = base_payroll + extra_payroll
constraints.append(total_payroll <= PAYROLL_CEILING)

# Acquisition Contraints:
MAX_FA_SIGNINGS = 6  # tweak this to taste
# Limit free agent binge
constraints.append(cp.sum(signed[is_fa]) <= MAX_FA_SIGNINGS)

# At most 1 QO signing from FA
constraints.append(cp.sum(cp.multiply(is_qo, signed)) <= 1)

# --- Trade realism: tiny budget + at most 1 trade ---

MAX_TRADE_BUDGET = 1.5  # small enough that Tier 2/3 depth is possible, but Tier 1 is impossible

# Only count trade_cost for actual trade acquisitions
constraints.append(cp.sum(cp.multiply(trade_cost, signed)) <= MAX_TRADE_BUDGET)

# At most 1 traded player total
idx_trades = np.where(is_trade)[0]
if len(idx_trades) > 0:
    constraints.append(cp.sum(signed[idx_trades]) <= 1)


unrealistic_trades = ["Adley Rutschman", "Tarik Skubal", "CJ Abrams", "Byron Buxton",
                      "Sonny Gray", "Brandon Nimmo", "Taylor Ward",]

mask_unrealistic = players["name"].isin(unrealistic_trades) & players["source"].eq("TRADE")
idx_unrealistic = np.where(mask_unrealistic.values)[0]
if len(idx_unrealistic) > 0:
    constraints.append(signed[idx_unrealistic] == 0)



# Objective: maximize total WAR of 26-man
objective = cp.Maximize(fg_war @ roster)

problem = cp.Problem(objective, constraints)
problem.solve(solver=cp.ECOS_BB)

print("Status:", problem.status)
print("Optimal projected WAR:", problem.value)

if problem.status not in ["optimal", "optimal_inaccurate"]:
    raise RuntimeError(f"Optimization did not find a feasible solution. Status: {problem.status}")


# --------------------------------------------------------
# 8. Extract and display the optimal 26-man roster
# --------------------------------------------------------

roster_sol = roster.value.round().astype(int)
signed_sol = signed.value.round().astype(int)

players["rostered"] = roster_sol
players["signed_flag"] = signed_sol

final_roster = players[players["rostered"] == 1].copy()
final_roster = final_roster.sort_values("fg_war", ascending=False)

final_war = final_roster["fg_war"].sum()

# extra payroll: only FAs + TRADE guys, whether or not they make 26-man
extra_payroll_val = float(np.dot(salary_extra, signed_sol))
total_payroll_val = base_payroll + extra_payroll_val

print("\n=== Optimal 26-man Roster (Opening Day) ===")
print(f"Total projected WAR: {final_war:.2f}")
print(f"Base Angels CBT payroll: ${base_payroll:,.0f}")
print(f"Added CBT from new acquisitions: ${extra_payroll_val:,.0f}")
print(f"Total CBT payroll: ${total_payroll_val:,.0f} (cap = ${PAYROLL_CEILING:,.0f})")

display_cols = [
    "name", "source", "pos", "fg_war", "salary"
]
print("\n", final_roster[display_cols].to_string(index=False))

Base Angels CBT payroll from roster file: $158,567,702
Status: optimal
Optimal projected WAR: 36.60869200005689

=== Optimal 26-man Roster (Opening Day) ===
Total projected WAR: 36.61
Base Angels CBT payroll: $158,567,702
Added CBT from new acquisitions: $49,321,667
Total CBT payroll: $207,889,369 (cap = $210,000,000)

              name source pos   fg_war     salary
        Zach Neto ANGELS  SS 3.615020  5200000.0
     Jose Soriano ANGELS  SP 3.014200  3250000.0
   Cody Bellinger     FA  CF 2.752690 26666667.0
    Yusei Kikuchi ANGELS  SP 2.532500 21225000.0
    Merrill Kelly     FA  SP 2.490130  9000000.0
Grayson Rodriguez ANGELS  SP 2.269910   820000.0
     Mark Vientos  TRADE  3B 2.081330   820000.0
   Nolan Schanuel ANGELS  1B 2.068450   820000.0
    Jorge Polanco     FA  2B 1.928450  5150000.0
     Reid Detmers ANGELS  SP 1.921450  2750000.0
         Jo Adell ANGELS  CF 1.669480  4500000.0
       Dustin May     FA  SP 1.502970  2135000.0
       Mike Trout ANGELS  RF 1.277660 355

In [3]:
# ============================================
# Build lineup, rotation, and bullpen from 26-man
# ============================================

import pandas as pd

# We'll work off final_roster (already filtered to rostered == 1)
lineup_pool = final_roster.copy()

# Make sure we have positional flags (in case they weren't printed)
pos_str = lineup_pool["pos"].astype(str)

if "is_c" not in lineup_pool.columns:
    lineup_pool["is_c"]  = pos_str.str.contains(r"\bC\b")
if "is_1b" not in lineup_pool.columns:
    lineup_pool["is_1b"] = pos_str.str.contains("1B")
if "is_2b" not in lineup_pool.columns:
    lineup_pool["is_2b"] = pos_str.str.contains("2B")
if "is_3b" not in lineup_pool.columns:
    lineup_pool["is_3b"] = pos_str.str.contains("3B")
if "is_ss" not in lineup_pool.columns:
    lineup_pool["is_ss"] = pos_str.str.contains("SS")
if "is_of" not in lineup_pool.columns:
    lineup_pool["is_of"] = pos_str.str.contains("LF|CF|RF")

# More granular OF flags
if "is_cf" not in lineup_pool.columns:
    lineup_pool["is_cf"] = pos_str.str.contains("CF")
if "is_lf" not in lineup_pool.columns:
    lineup_pool["is_lf"] = pos_str.str.contains("LF")
if "is_rf" not in lineup_pool.columns:
    lineup_pool["is_rf"] = pos_str.str.contains("RF")

if "is_pitcher" not in lineup_pool.columns:
    lineup_pool["is_pitcher"] = pos_str.str.contains("SP|RP")
if "is_sp" not in lineup_pool.columns:
    lineup_pool["is_sp"] = pos_str.str.contains("SP")
if "is_rp" not in lineup_pool.columns:
    lineup_pool["is_rp"] = pos_str.str.contains("RP") & ~lineup_pool["is_sp"]


def pick_best(mask, used_names, desc):
    """Pick best fg_war player satisfying mask and not already used."""
    cand = lineup_pool[mask & ~lineup_pool["name"].isin(used_names)]
    if cand.empty:
        # No one fits; return None row
        return None
    return cand.sort_values("fg_war", ascending=False).iloc[0]


# --------------------------------------------
# 1) Starting lineup vs RHP (9 players)
# --------------------------------------------

used = []

lineup_slots = []

# 1B, 2B, SS, 3B
slot_defs = [
    ("1B",  lambda df: df["is_1b"]),
    ("2B",  lambda df: df["is_2b"]),
    ("SS",  lambda df: df["is_ss"]),
    ("3B",  lambda df: df["is_3b"]),
]

# C: prefer Logan O'Hoppe if present, else best C
c_row = None
oho_mask = (lineup_pool["name"] == "Logan O'Hoppe") & lineup_pool["is_c"]
c_row = pick_best(oho_mask, used, "C (O'Hoppe first)")
if c_row is None:
    c_row = pick_best(lineup_pool["is_c"], used, "C")

if c_row is not None:
    used.append(c_row["name"])
    lineup_slots.append({"order": None, "field_pos": "C", "name": c_row["name"], "pos": c_row["pos"], "fg_war": c_row["fg_war"]})

# IF slots
for label, mask_fn in slot_defs:
    row = pick_best(mask_fn(lineup_pool), used, label)
    if row is not None:
        used.append(row["name"])
        lineup_slots.append({"order": None, "field_pos": label, "name": row["name"], "pos": row["pos"], "fg_war": row["fg_war"]})

# OF: CF, then corners
# CF: prefer an external CF if you want, but here just best true CF
cf_row = pick_best(lineup_pool["is_cf"], used, "CF")
if cf_row is None:
    # fallback: any OF as CF if no true CF
    cf_row = pick_best(lineup_pool["is_of"], used, "CF (fallback)")

if cf_row is not None:
    used.append(cf_row["name"])
    lineup_slots.append({"order": None, "field_pos": "CF", "name": cf_row["name"], "pos": cf_row["pos"], "fg_war": cf_row["fg_war"]})

# Corner OF: LF + RF (or generic OF if needed)
lf_row = pick_best(lineup_pool["is_lf"], used, "LF")
if lf_row is None:
    lf_row = pick_best(lineup_pool["is_of"], used, "LF (fallback)")
if lf_row is not None:
    used.append(lf_row["name"])
    lineup_slots.append({"order": None, "field_pos": "LF", "name": lf_row["name"], "pos": lf_row["pos"], "fg_war": lf_row["fg_war"]})

rf_row = pick_best(lineup_pool["is_rf"], used, "RF")
if rf_row is None:
    rf_row = pick_best(lineup_pool["is_of"], used, "RF (fallback)")
if rf_row is not None:
    used.append(rf_row["name"])
    lineup_slots.append({"order": None, "field_pos": "RF", "name": rf_row["name"], "pos": rf_row["pos"], "fg_war": rf_row["fg_war"]})

# DH: prefer in-house DH types (Soler, then Trout), then best remaining bat
dh_priority = ["Jorge Soler", "Mike Trout"]

# candidates: non-pitchers not already used
dh_cands = lineup_pool[(~lineup_pool["is_pitcher"]) & ~lineup_pool["name"].isin(used)].copy()

dh_row = None
for pname in dh_priority:
    cand = dh_cands[dh_cands["name"] == pname]
    if not cand.empty:
        dh_row = cand.sort_values("fg_war", ascending=False).iloc[0]
        break

# if neither Soler nor Trout is available for DH, fall back to best bat
if dh_row is None and not dh_cands.empty:
    dh_row = dh_cands.sort_values("fg_war", ascending=False).iloc[0]

if dh_row is not None:
    used.append(dh_row["name"])
    lineup_slots.append({
        "order": None,
        "field_pos": "DH",
        "name": dh_row["name"],
        "pos": dh_row["pos"],
        "fg_war": dh_row["fg_war"],
    })

# Turn into DataFrame and assign batting order by WAR (best → 3/4 spots, etc.)
lineup_df = pd.DataFrame(lineup_slots)
lineup_df = lineup_df.sort_values("fg_war", ascending=False).reset_index(drop=True)
lineup_df["order"] = lineup_df.index + 1  # simple 1–9 by WAR; you can re-order manually later

lineup_df = lineup_df[["order", "field_pos", "name", "pos", "fg_war"]]

print("\n=== Starting Lineup vs RHP (auto-generated) ===")
print(lineup_df.to_string(index=False))

# Bench = hitters not in starting lineup
starter_names = set(lineup_df["name"])
bench_pool = lineup_pool[~lineup_pool["is_pitcher"] & ~lineup_pool["name"].isin(starter_names)].copy()
bench_pool = bench_pool.sort_values("fg_war", ascending=False).reset_index(drop=True)

# Simple bench role heuristics
bench_roles = []

for _, row in bench_pool.iterrows():
    name = row["name"]
    pos = row["pos"]
    is_c = row["is_c"]
    is_of = row["is_of"]
    is_inf = row["is_inf"]
    is_ss = row["is_ss"]

    role = "Bench Bat"

    if is_c:
        role = "C2 / Backup C"
    elif is_of and is_inf:
        role = "Super Utility"
    elif is_of:
        role = "4th/5th OF"
    elif is_inf and is_ss:
        role = "Utility SS/MI"
    elif is_inf:
        role = "Utility IF"

    bench_roles.append(role)

bench_pool["bench_role"] = bench_roles

print("\n=== Bench (Position Players) ===")
print(bench_pool[["name", "pos", "fg_war", "bench_role"]].to_string(index=False))


# --------------------------------------------
# 2) 5-man rotation
# --------------------------------------------

sp_pool = lineup_pool[lineup_pool["is_sp"]].sort_values("fg_war", ascending=False)
rotation_df = sp_pool.head(5).copy()
rotation_df["rotation_spot"] = ["SP1", "SP2", "SP3", "SP4", "SP5"][: len(rotation_df)]

print("\n=== 5-Man Rotation ===")
print(rotation_df[["rotation_spot", "name", "pos", "fg_war"]].to_string(index=False))


# --------------------------------------------
# 3) Bullpen (everyone else who is a pitcher)
# --------------------------------------------

rotation_names = set(rotation_df["name"])
bp_pool = lineup_pool[lineup_pool["is_pitcher"] & ~lineup_pool["name"].isin(rotation_names)].copy()
bp_pool = bp_pool.sort_values("fg_war", ascending=False).reset_index(drop=True)

# Cap bullpen at 7 arms (5-man rotation + 7-man pen)
bp_pool = bp_pool.head(7).copy()

role_labels = ["Late/High-Leverage", "Setup", "Middle", "Middle", "Long/Bulk", "Long/Bulk", "Depth"]
bp_pool["role_hint"] = role_labels[: len(bp_pool)]
# Override bullpen roles for specific guys (Angels-specific view)
bp_overrides = {
    "Ben Joyce": "Closer",
    "Robert Stephenson": "Setup",
    "Dustin May": "Bulk / High-Leverage",
    "Adrian Houser": "Long Relief / Spot Starter",
    "Chris Paddack": "Middle / Multi-Inning",
    "Ryan Johnson": "Middle Relief",
    "Chase Silseth": "Depth / Swingman",
}

bp_pool["role_hint"] = bp_pool.apply(
    lambda row: bp_overrides.get(row["name"], row["role_hint"]),
    axis=1
)
print("\n=== Bullpen ===")
print(bp_pool[["name", "pos", "fg_war", "role_hint"]].to_string(index=False))




=== Starting Lineup vs RHP (auto-generated) ===
 order field_pos           name pos   fg_war
     1        SS      Zach Neto  SS 3.615020
     2        CF Cody Bellinger  CF 2.752690
     3        3B   Mark Vientos  3B 2.081330
     4        1B Nolan Schanuel  1B 2.068450
     5        2B  Jorge Polanco  2B 1.928450
     6        LF       Jo Adell  CF 1.669480
     7        RF     Mike Trout  RF 1.277660
     8         C  Logan O'Hoppe   C 0.876898
     9        DH    Jorge Soler  RF 0.770061

=== Bench (Position Players) ===
           name pos   fg_war    bench_role
Christian Moore  2B 1.054510    Utility IF
  Oswald Peraza  SS 0.358929 Utility SS/MI
Travis d'Arnaud   C 0.317418 C2 / Backup C
 Anthony Rendon  3B 0.300777    Utility IF

=== 5-Man Rotation ===
rotation_spot              name pos  fg_war
          SP1      Jose Soriano  SP 3.01420
          SP2     Yusei Kikuchi  SP 2.53250
          SP3     Merrill Kelly  SP 2.49013
          SP4 Grayson Rodriguez  SP 2.26991
        